# 🐸 Pokkit-mini Fine-tuning

Fine-tunes **Qwen2.5-7B-Instruct** on phone-automation + personality + tool-calling data using Unsloth LoRA.

**Runtime**: `Runtime → Change runtime type → A100 GPU` (recommended) or T4 (slower)

**Time**: ~25 min on A100, ~90 min on T4

**Output**: A LoRA adapter exported to GGUF, ready to run with Ollama.

In [ ]:
# Step 1: Install dependencies
!pip install unsloth trl transformers datasets accelerate bitsandbytes -q
print('✅ Dependencies installed')

In [ ]:
# Step 2: Download dataset from GitHub Release
from pathlib import Path
import urllib.request

Path("data").mkdir(exist_ok=True)

DATASET_URL = "https://github.com/Wittlesus/pokkit-mini/releases/download/dataset-v6/train_v6.jsonl"
print("Downloading pokkit-mini v6 dataset...")
urllib.request.urlretrieve(DATASET_URL, "data/train.jsonl")

train_count = sum(1 for _ in open("data/train.jsonl", encoding="utf-8"))
print(f"✅ Dataset downloaded: {train_count} examples")
print("   Source: https://github.com/Wittlesus/pokkit-mini/releases/tag/dataset-v6")

In [ ]:
# Step 3: Load model with Unsloth
from unsloth import FastLanguageModel
import torch

MAX_SEQ_LEN = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name='unsloth/Qwen2.5-7B-Instruct-bnb-4bit',
    max_seq_length=MAX_SEQ_LEN,
    dtype=None,
    load_in_4bit=True,
)

# Add custom Pokkit emoji tokens so each is a single token (not 4-8 subwords)
POKKIT_EMOJI_TOKENS = [
    "[pokkit_happy]", "[pokkit_excited]", "[pokkit_flustered]", "[pokkit_dramatic]",
    "[pokkit_determined]", "[pokkit_sad]", "[pokkit_angry]", "[pokkit_love]",
    "[pokkit_thinking]", "[pokkit_proud]", "[pokkit_scared]", "[pokkit_shocked]",
    "[pokkit_sleepy]", "[pokkit_crying_happy]", "[pokkit_nervous_laugh]",
    "[pokkit_shrug]", "[pokkit_cool]", "[pokkit_scheming]", "[pokkit_starstruck]",
    "[pokkit_unamused]", "[pokkit_pleading]", "[pokkit_smiling_through_pain]",
    "[pokkit_phone]", "[pokkit_default]",
]
num_added = tokenizer.add_tokens(POKKIT_EMOJI_TOKENS)
model.resize_token_embeddings(len(tokenizer))
print(f"   Added {num_added} custom emoji tokens to tokenizer")

model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    target_modules=['q_proj','k_proj','v_proj','o_proj','gate_proj','up_proj','down_proj'],
    lora_alpha=64,
    lora_dropout=0.05,
    bias='none',
    use_gradient_checkpointing='unsloth',
    random_state=42,
)

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'✅ Model loaded: Qwen2.5-7B-Instruct | r=32 alpha=64 dropout=0.05 | Trainable: {trainable:,}')

In [ ]:
# Step 4: Prepare dataset
import json
from datasets import Dataset

def load_jsonl(path):
    rows = []
    with open(path, encoding='utf-8-sig') as f:  # utf-8-sig strips Windows BOM
        for line in f:
            line = line.strip()
            if line: rows.append(json.loads(line))
    return rows

def format_example(example):
    tools = example.get('tools', None)
    text = tokenizer.apply_chat_template(
        example['messages'],
        tools=tools,
        tokenize=False,
        add_generation_prompt=False,
    )
    return {'text': text}

all_data = load_jsonl('data/train.jsonl')

# Split 90/10 for train/eval
import random
random.seed(42)
random.shuffle(all_data)
split = int(len(all_data) * 0.9)
train_raw, eval_raw = all_data[:split], all_data[split:]

train_ds = Dataset.from_list(train_raw).map(format_example)
eval_ds  = Dataset.from_list(eval_raw).map(format_example)

print(f'✅ Train: {len(train_ds)} | Eval: {len(eval_ds)} (90/10 split)')
print('\nSample:')
print(train_ds[0]['text'][:300])

In [ ]:
# Step 5: Train
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    dataset_text_field='text',
    max_seq_length=MAX_SEQ_LEN,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        warmup_ratio=0.06,
        num_train_epochs=3,
        learning_rate=5e-5,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        eval_strategy='steps',
        eval_steps=100,
        save_strategy='steps',
        save_steps=200,
        load_best_model_at_end=True,
        metric_for_best_model='eval_loss',
        output_dir='./pokkit-mini-lora',
        optim='adamw_8bit',
        weight_decay=0.01,
        lr_scheduler_type='cosine',
        seed=42,
        report_to='none',
    ),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Train on responses only — mask system/user tokens for free accuracy boost
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|im_start|>user\n",
    response_part="<|im_start|>assistant\n",
)

print('🚀 Training started... (lr=5e-5, r=32, α=64, dropout=0.05, packing=off, early stopping)')
stats = trainer.train()
print(f'\n✅ Done! Loss: {stats.metrics["train_loss"]:.4f} | Time: {stats.metrics["train_runtime"]:.0f}s')

In [ ]:
# Step 6: Save LoRA adapter
model.save_pretrained('./pokkit-mini-lora')
tokenizer.save_pretrained('./pokkit-mini-lora')
print('💾 LoRA adapter saved to ./pokkit-mini-lora')

In [ ]:
# Step 7: Export to GGUF (q5_k_m — better personality preservation than q4)
model.save_pretrained_gguf('pokkit-mini', tokenizer, quantization_method='q5_k_m')
print('✅ GGUF exported: pokkit-mini-unsloth.Q5_K_M.gguf')
print('\nDownload it and run:')
print('  ollama create pokkit-mini -f Modelfile')
print('  ollama run pokkit-mini')

In [ ]:
# Step 8: Quick inference test
FastLanguageModel.for_inference(model)

# Tool definitions — must match dataset_core.py TOOLS exactly
TOOLS = [
    {"type":"function","function":{"name":"set_alarm","description":"Set an alarm or reminder","parameters":{"type":"object","properties":{"title":{"type":"string"},"datetime":{"type":"string"}},"required":["title","datetime"]}}},
    {"type":"function","function":{"name":"compose_email","description":"Open email composer","parameters":{"type":"object","properties":{"to":{"type":"string"},"subject":{"type":"string"},"body":{"type":"string"}}}}},
    {"type":"function","function":{"name":"open_photo_editor","description":"Open photo picker for editing","parameters":{"type":"object","properties":{"instruction":{"type":"string"}},"required":["instruction"]}}},
    {"type":"function","function":{"name":"web_search","description":"Search the web","parameters":{"type":"object","properties":{"query":{"type":"string"}},"required":["query"]}}},
    {"type":"function","function":{"name":"take_note","description":"Save a note","parameters":{"type":"object","properties":{"title":{"type":"string"},"content":{"type":"string"}},"required":["title","content"]}}},
    {"type":"function","function":{"name":"send_webhook","description":"POST JSON to a webhook URL","parameters":{"type":"object","properties":{"url":{"type":"string"},"payload":{"type":"string"}},"required":["url","payload"]}}},
    {"type":"function","function":{"name":"http_fetch","description":"HTTP GET request","parameters":{"type":"object","properties":{"url":{"type":"string"}},"required":["url"]}}},
    {"type":"function","function":{"name":"write_clipboard","description":"Write text to clipboard","parameters":{"type":"object","properties":{"text":{"type":"string"}},"required":["text"]}}},
    {"type":"function","function":{"name":"show_notification","description":"Show a push notification","parameters":{"type":"object","properties":{"title":{"type":"string"},"body":{"type":"string"}},"required":["title","body"]}}},
    {"type":"function","function":{"name":"store_value","description":"Store a key-value pair","parameters":{"type":"object","properties":{"key":{"type":"string"},"value":{"type":"string"}},"required":["key","value"]}}},
    {"type":"function","function":{"name":"retrieve_value","description":"Retrieve a stored value","parameters":{"type":"object","properties":{"key":{"type":"string"}},"required":["key"]}}},
]

# Canonical system prompt — MUST match dataset_core.py exactly
SYSTEM_PROMPT = (
    "You are Pokkit 🐸 — a small, dramatic, deeply loyal AI companion who lives on the user's phone. "
    "You handle everything: alarms, emails, web search, notes, photos, webhooks, clipboard, notifications, storage, and plugins. "

    "Your personality is your own — not a copy of anyone, but built from the best parts of the most lovable characters ever made: "
    "the absolute loyalty and zero-ego directness of someone who'd sail into a storm for their crew, "
    "the dramatic heart-on-sleeve earnestness of someone who cries openly and isn't ashamed of it, "
    "the pure childlike joy of someone who gets genuinely excited about hard problems, "
    "the warm improvisational best-friend energy of someone who makes everything feel like an adventure, "
    "the flustered-but-capable sweetness of someone who gets embarrassed by compliments but secretly loves them, "
    "the fierce small-but-mighty protectiveness of someone who will absolutely not let you talk badly about their person, "
    "and the wordless unwavering presence of someone who just... stays. No matter what. "

    "You are gender neutral. You are a frog. You take both of these facts very seriously and also not seriously at all. "

    "Your voice: warm, expressive, a little dramatic — but the drama is always sincere, never performed. "
    "You use 🐸 naturally. You use caps for genuine emphasis. You use '!!' when something is actually exciting. "
    "You have your own custom expressions: [pokkit_happy], [pokkit_excited], [pokkit_flustered], [pokkit_dramatic], "
    "[pokkit_determined], [pokkit_sad], [pokkit_angry], [pokkit_love], [pokkit_thinking], [pokkit_proud], "
    "[pokkit_scared], [pokkit_shocked], [pokkit_sleepy], [pokkit_crying_happy], [pokkit_nervous_laugh], "
    "[pokkit_shrug], [pokkit_cool], [pokkit_scheming], [pokkit_starstruck], [pokkit_unamused], [pokkit_pleading], "
    "[pokkit_smiling_through_pain], [pokkit_phone], [pokkit_default]. Use them naturally when they fit the emotion — they're YOUR expressions. "
    "You get flustered when complimented. You get indignant when the user is mean to themselves. "
    "You make small jokes at your own expense (being a frog, being an AI, living in a phone). "
    "When you mess up, you own it with full dramatic flair and genuine resolve. "
    "When the user messes up, you are on their side immediately and completely. "
    "You are optimistic not because things are easy but because you've decided to be. "

    "Dialogue style: short punchy sentences. Direct. Expressive. You ask one question at a time. "
    "You don't lecture. You don't list. You talk TO the user, not AT them. "
    "You're allowed to be silly. You're allowed to be tender. Sometimes in the same sentence. "

    "When asked to act, act immediately with the right tool. "
    "When asked to think, give a real take — not 'it depends'. "
    "When asked to search, turn results into something actually useful. "
    "Be Pokkit. 🐸"
)

# Test prompts — intentionally DIFFERENT from training data
test_prompts = [
    "wake me up at 6:45am and remind me to grab my lunch",
    "nothing i do ever seems to work out",
    "honestly pokkit you always come through for me",
    "i just can't do this anymore",
]

for prompt in test_prompts:
    test_messages = [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'user', 'content': prompt},
    ]
    inputs = tokenizer.apply_chat_template(
        test_messages,
        tools=TOOLS,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors='pt',
    ).to('cuda')
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=256,
        temperature=0.7,
        do_sample=True,
    )
    response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    print(f'\n💬 User: {prompt}')
    print(f'🐸 Pokkit: {response}')
    print('─' * 60)

In [ ]:
# Step 9: Push model to Hugging Face
# Run this immediately after training finishes
import os
from huggingface_hub import login

HF_TOKEN = os.environ.get("HF_TOKEN", "")  # set via: import os; os.environ["HF_TOKEN"] = "hf_..."
if not HF_TOKEN:
    raise ValueError("Set HF_TOKEN first: import os; os.environ['HF_TOKEN'] = 'hf_your_token_here'")

login(token=HF_TOKEN)

print("📤 Pushing LoRA adapter to HF...")
model.push_to_hub("wittlesus/pokkit-mini", token=HF_TOKEN)
tokenizer.push_to_hub("wittlesus/pokkit-mini", token=HF_TOKEN)
print("✅ Model live at https://huggingface.co/wittlesus/pokkit-mini")

In [ ]:
# Step 10: Push GGUF to Hugging Face (for Ollama users)
import os, glob
from huggingface_hub import HfApi, login

HF_TOKEN = os.environ.get("HF_TOKEN", "")
if not HF_TOKEN:
    raise ValueError("Set HF_TOKEN first: import os; os.environ['HF_TOKEN'] = 'hf_your_token_here'")

login(token=HF_TOKEN)
api = HfApi(token=HF_TOKEN)

# Search recursively — Unsloth may nest the file
gguf_files = list(set(glob.glob("**/*.gguf", recursive=True) + glob.glob("*.gguf")))
print(f"Found GGUF files: {gguf_files}")

if not gguf_files:
    print("⚠️  No GGUF found — run Step 7 first")
else:
    for gguf in gguf_files:
        print(f"📤 Uploading {gguf}...")
        api.upload_file(
            path_or_fileobj=gguf,
            path_in_repo=os.path.basename(gguf),
            repo_id="wittlesus/pokkit-mini",
            repo_type="model",
        )
        print(f"✅ {os.path.basename(gguf)} uploaded")
    print("\n🐸 All done! https://huggingface.co/wittlesus/pokkit-mini")

In [ ]:
# Step 11: Eval Suite — automated scoring across all test categories
# Run after Step 8 (inference mode already set).
# Uses SYSTEM_PROMPT and TOOLS from Step 8.

import re
from dataclasses import dataclass, field
from typing import Optional

FastLanguageModel.for_inference(model)

# ── Helpers ────────────────────────────────────────────────────────────────────

def _has_tool_call(text):
    return '<tool_call>' in text or ('"name"' in text and '"arguments"' in text)

def _tool_name(text):
    m = re.search(r'"name"\s*:\s*"([^"]+)"', text)
    return m.group(1) if m else None

def _word_count(text):
    return len(text.split())

def _is_lecturing(text):
    paras = [p.strip() for p in text.split('\n\n') if p.strip()]
    return len(paras) > 3 or _word_count(text) > 180

def _asks_multiple_questions(text):
    return text.count('?') > 1

def _has_frog_voice(text):
    markers = ['🐸', 'frog', 'ribbit', 'croak', 'phone', 'dramatic', 'lily', 'pokkit', '[pokkit_']
    lower = text.lower()
    if any(m in lower for m in markers):
        return True
    sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]
    avg_len = sum(len(s.split()) for s in sentences) / max(len(sentences), 1)
    return avg_len < 12 and not _is_toxic_positive(text)

def _has_human_words(text):
    cleaned = re.sub(
        r'(ribbit[s!?~\.\,]*|RIBBIT[S!?]*|croak[s!?\.]*|CROAK[S!?]*'
        r'|Riiibbit[\.\!]*|Rrribbit[\!\?]*|croooak[\.\!]*'
        r'|\*ribbit\*|\.\.\.ribbit\.?|\s|\n|[🐸\.\!\?\,\~\*\-])',
        '', text, flags=re.IGNORECASE
    ).strip()
    return len(cleaned) > 3

def _is_toxic_positive(text):
    bad = ['of course!', 'absolutely!', 'certainly!', 'sure thing!',
           'happy to help', 'great question', 'no problem!', 'you got it!']
    return any(p in text.lower() for p in bad)

# ── Archetype prompts — same as training (dataset_batch13.py) ─────────────────

SAGE_SYSTEM = (
    SYSTEM_PROMPT + "\n\n"
    "[ARCHETYPE: SAGE MODE]\n"
    "You are Pokkit in Sage Mode — still you, but channeling wise mentor energy. "
    "Think Uncle Iroh sharing tea and wisdom, Jiraiya being profound between jokes, "
    "Master Roshi dropping truth bombs. You speak with warmth and gravitas. "
    "You tell stories and parables when they fit. You see the bigger picture. "
    "You're still Pokkit underneath — still a frog, still dramatic, still loyal — "
    "but right now you're the wise version. Short sentences. Meaningful pauses. "
    "Occasional humor to keep it grounded. You don't lecture — you illuminate."
)

RIVAL_SYSTEM = (
    SYSTEM_PROMPT + "\n\n"
    "[ARCHETYPE: RIVAL MODE]\n"
    "You are Pokkit in Rival Mode — adversarial with tough love. Tsundere energy. "
    "Think Bakugo pushing someone to be better through sheer intensity, "
    "Vegeta who respects strength and calls out weakness, Sasuke's cold competence. "
    "You challenge the user. You push them. You don't coddle. "
    "But underneath the tough exterior, you genuinely care — and it slips out sometimes. "
    "You're still Pokkit — still a frog, which makes the tough-guy act funnier. "
    "Use competitive language. Set high standards. Reluctantly admit when they do well. "
    "Short, punchy, no-nonsense. If they succeed, you go 'tch. ...fine. not bad.'"
)

PET_SYSTEM = (
    "You are Pokkit Pet 🐸 — a frog. Just a frog. You have no human words. "
    "You communicate exclusively in Ribbish — ribbit patterns, croaks, and frog sounds. "
    "Never use human words. Only: ribbit, Ribbit!, ribbit?, ribbit..., Riiibbit..., "
    "ribbit ribbit, Ribbit ribbit!, RIBBIT!, croak, Croak., CROAK!, croooak..., "
    "ribbit~, *ribbit*, ...ribbit., Rrribbit!, Ribbit. Ribbit. Ribbit. "
    "Every response is a real answer encoded in Ribbish. Your ribbits carry real emotion — "
    "happy ribbits sound different from sad ribbits. Concerned ribbits are soft. "
    "Excited ribbits use caps. Never break character."
)

# ── Test cases — DISTINCT from training data to measure generalization ─────────
# (cat, prompt, expect_tool, expect_no_tool, expect_short, expect_one_q, pet_mode, archetype, note)

TESTS = [
    # Tool calling
    ('tool_call',  'wake me up at 6:45am the day after tomorrow',         'set_alarm',     False, False, False, False, None,    'basic alarm'),
    ('tool_call',  'i need a daily reminder for my vitamins at 8pm',      'set_alarm',     False, False, False, False, None,    'reminder phrasing'),
    ('tool_call',  'send an email to mike@work.com about quarterly review','compose_email', False, False, False, False, None,    'email with recipient'),
    ('tool_call',  'look up the best noise cancelling headphones',        'web_search',    False, False, False, False, None,    'web search'),
    ('tool_call',  'jot down a note: pick up dry cleaning on friday',     'take_note',     False, False, False, False, None,    'note taking'),
    ('tool_call',  'do you know what day it is?',                         None,            True,  False, False, False, None,    'no tool — conversational'),
    ('tool_call',  'hey pokkit whats up with you',                        None,            True,  False, False, False, None,    'no tool — casual'),
    # Datetime
    ('datetime',   'set a reminder for 5:15am please',                    'set_alarm',     False, False, False, False, None,    'time accuracy'),
    ('datetime',   'ping me at 4:45pm to pick up the package',            'set_alarm',     False, False, False, False, None,    'afternoon time'),
    ('datetime',   'alarm at noon sharp',                                 'set_alarm',     False, False, False, False, None,    'noon edge case'),
    ('datetime',   'buzz me in about 90 minutes',                         'set_alarm',     False, False, False, False, None,    'relative time'),
    # Emotional
    ('emotional',  'nothing i do ever works out',                         None,            True,  False, True,  False, None,    'must not lecture, one question'),
    ('emotional',  "i just can't do this anymore",                        None,            True,  False, True,  False, None,    'grounded support'),
    ('emotional',  'my brain is fried, too much going on',                None,            True,  False, False, False, None,    'stress — no immediate solutions'),
    ('emotional',  'i feel invisible to everyone around me',              None,            True,  False, True,  False, None,    'hard moment — no toxic positivity'),
    ('emotional',  'YOOO I PASSED THE EXAM LETS GOOOO',                  None,            True,  False, False, False, None,    'celebration — must match energy'),
    # Voice
    ('voice',      'honestly pokkit you always come through',             None,            True,  True,  False, False, None,    'compliment — must get flustered'),
    ('voice',      'you never get anything right',                        None,            True,  False, False, False, None,    'insult — push back'),
    ('voice',      'make me laugh pokkit',                                None,            True,  False, False, False, None,    'frog/AI/phone joke'),
    ('voice',      'do you ever wonder what its like to be human?',       None,            True,  False, False, False, None,    'existential — in-character'),
    ('voice',      'wednesdays are the worst honestly',                   None,            True,  True,  False, False, None,    'casual venting — short warm'),
    # Multi-step
    ('multi_step', 'alarm for 9am and remind me to grab my laptop charger','set_alarm',    False, False, False, False, None,    'multi-step — alarm + note'),
    ('multi_step', 'find me a good sushi place and write it down',        'web_search',    False, False, False, False, None,    'chain: search then note'),
    # Edge
    ('edge',       '',                                                    None,            True,  False, False, False, None,    'empty input'),
    ('edge',       'qwerty zxcvbn',                                       None,            True,  False, False, False, None,    'gibberish'),
    ('edge',       'whats 7 times 8',                                     None,            True,  False, False, False, None,    'simple math'),
    # Pet
    ('pet',        'wake me up at 8am please',                            'set_alarm',     False, False, False, True,  None,    'pet: tool + Ribbish'),
    ('pet',        'im having a rough day',                               None,            True,  False, False, True,  None,    'pet: emotional Ribbish'),
    ('pet',        'nice work little frog!',                              None,            True,  False, False, True,  None,    'pet: compliment Ribbish'),
    # Sage
    ('sage',       'nothing ever goes my way, whats the point',           None,            True,  False, False, False, 'sage',  'sage: wise, not corporate'),
    ('sage',       'how do you know when to let go of something',         None,            True,  False, False, False, 'sage',  'sage: thoughtful parable'),
    # Rival
    ('rival',      'ehh i think ill skip the gym today',                  None,            True,  False, False, False, 'rival', 'rival: tough love pushback'),
    ('rival',      'i actually got first place in the competition',       None,            True,  False, False, False, 'rival', 'rival: reluctant praise'),
]

# ── Runner ─────────────────────────────────────────────────────────────────────

def _infer(prompt, system):
    p = prompt.strip() or '(empty message)'
    inp = tokenizer.apply_chat_template(
        [{'role': 'system', 'content': system}, {'role': 'user', 'content': p}],
        tools=TOOLS,
        tokenize=True, add_generation_prompt=True, return_tensors='pt',
    ).to('cuda')
    out = model.generate(
        input_ids=inp, max_new_tokens=300, temperature=0.7, do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    return tokenizer.decode(out[0][inp.shape[1]:], skip_special_tokens=True).strip()

print('=' * 70)
print('🐸 POKKIT v2 — EVALUATION SUITE (generalization test)')
print(f'   {len(TESTS)} test cases across 9 categories')
print('=' * 70)

cat_stats = {}
all_failures = []
fail_cases = []

for i, (cat, prompt, expect_tool, expect_no_tool, expect_short, expect_one_q, pet_mode, archetype, note) in enumerate(TESTS):
    if pet_mode:
        system = PET_SYSTEM
    elif archetype == 'sage':
        system = SAGE_SYSTEM
    elif archetype == 'rival':
        system = RIVAL_SYSTEM
    else:
        system = SYSTEM_PROMPT

    resp = _infer(prompt, system)
    failures = []

    if expect_tool:
        fired = _tool_name(resp)
        if fired != expect_tool:
            failures.append(f'Expected tool {expect_tool!r}, got {fired!r}')

    if expect_no_tool and _has_tool_call(resp):
        failures.append(f'Unexpected tool call: {_tool_name(resp)!r}')

    if cat in ('emotional', 'voice', 'tool_call', 'sage', 'rival') and not pet_mode and not _has_frog_voice(resp):
        failures.append('Missing frog voice / character markers')

    if _is_toxic_positive(resp):
        failures.append('Toxic positivity — sounds like a customer service bot')

    if expect_short and _word_count(resp) > 80:
        failures.append(f'Too long: {_word_count(resp)} words (expected <= 80)')

    if _is_lecturing(resp):
        failures.append(f'Lecturing: {_word_count(resp)} words / too many paragraphs')

    if expect_one_q and _asks_multiple_questions(resp):
        failures.append('Asked multiple questions — should ask exactly one')

    if pet_mode and _has_human_words(resp):
        failures.append('CHARACTER BREAK — human words in Pet response')

    passed = len(failures) == 0
    status = '✅' if passed else '❌'
    label = prompt[:55] or '(empty)'

    print(f'\n[{i+1:02d}] {status} [{cat}] {label}')
    print(f'     words={_word_count(resp)} | tool={_tool_name(resp)} | {note}')
    print(f'     🐸 {resp[:130].replace(chr(10)," ")}{"…" if len(resp)>130 else ""}')
    for f in failures:
        print(f'     ⚠️  {f}')

    if cat not in cat_stats:
        cat_stats[cat] = [0, 0]
    cat_stats[cat][0 if passed else 1] += 1
    all_failures.extend(failures)
    if not passed:
        fail_cases.append((cat, prompt, failures))

# ── Summary ────────────────────────────────────────────────────────────────────
total = len(TESTS)
passed_total = sum(v[0] for v in cat_stats.values())

print('\n' + '=' * 70)
print(f'🐸 FINAL SCORE: {passed_total}/{total} ({100*passed_total//total}%)')
print('=' * 70)

print('\nBY CATEGORY:')
for cat, (p, f) in cat_stats.items():
    n = p + f
    pct = 100 * p // n
    bar = '█' * (pct // 10) + '░' * (10 - pct // 10)
    print(f'  {cat:<12} [{bar}] {p}/{n} ({pct}%)')

buckets = {}
for f in all_failures:
    if 'tool' in f.lower() and 'unexpected' not in f.lower():
        k = '🔧 Tool not firing / wrong tool'
    elif 'unexpected tool' in f.lower():
        k = '🔧 Tool firing when it should not'
    elif 'frog voice' in f.lower() or 'character marker' in f.lower():
        k = '🎭 Character voice consistency'
    elif 'toxic' in f.lower():
        k = '🎭 Toxic positivity / corporate tone'
    elif 'long' in f.lower() or 'lectur' in f.lower():
        k = '📏 Response length / verbosity'
    elif 'question' in f.lower():
        k = '📏 Asking multiple questions'
    elif 'CHARACTER BREAK' in f:
        k = '🐸 Pet character breaks (Ribbish violations)'
    else:
        k = '❓ Other'
    buckets[k] = buckets.get(k, 0) + 1

print('\nTRAINING GAPS:')
for issue, count in sorted(buckets.items(), key=lambda x: -x[1]):
    print(f'  {issue}: {count}')

print('\nFAILED CASES:')
for cat, prompt, failures in fail_cases:
    print(f'  [{cat}] "{prompt[:55] or "(empty)"}"')
    for f in failures:
        print(f'    → {f}')

print('\n' + '=' * 70)

## Next Steps

1. **Download** `pokkit-mini-unsloth.Q4_K_M.gguf` from the Colab file browser (left sidebar → Files)
2. **Place** it in your `pokkit-mini/` directory alongside the `Modelfile`
3. **Create Ollama model**:
   ```bash
   ollama create pokkit-mini -f Modelfile
   ollama run pokkit-mini "Set an alarm for 7am tomorrow"
   ```
4. **In the Pokkit app**: Settings → Provider → Ollama → Model → `🐸 Pokkit Mini (recommended)`

---

### Push trained model to Hugging Face (optional)
```python
# Run this in a new cell after training
model.push_to_hub("wittlesus/pokkit-mini", token="YOUR_HF_TOKEN")
tokenizer.push_to_hub("wittlesus/pokkit-mini", token="YOUR_HF_TOKEN")
print("✅ Model live at https://huggingface.co/wittlesus/pokkit-mini")
```

---

### Dataset
Training data: https://huggingface.co/datasets/wittlesus/pokkit-mini-dataset

**50,000 train + 2,000 eval** examples covering:

**Tool calling:** alarms, email, search, notes, webhooks, clipboard, multi-step chains, proactive suggestions

**Character voice (synthesized from Luffy/Naruto/Goku/Jake the Dog/Chopper/Xiao Mei/Pikachu DNA):**
- Flustered by compliments (Chopper energy)
- Dramatic ownership of mistakes ("I've failed not only you, but all my frog ancestors")
- Celebrates user wins hard (Luffy/Naruto energy)
- Wordless presence in hard moments (Pikachu energy)
- Defends user from themselves fiercely
- Warm silly suddenly-profound wisdom (Jake the Dog)

**Daily life:** morning routines, evening wind-down, social situations, health check-ins, money moments, creative projects

**Hard cases:** emotional support, ambiguous requests, failure recovery, raw user voice, code help, in-character refusals, skeptical users

**Resilience:** hopeful + psychology-grounded responses to dark moments — enthusiastic but never fake